In [18]:
import json
import os
import datetime
import math
from pymongo import MongoClient
from bson import json_util, ObjectId
import json
from pymongo import MongoClient
from bson import json_util

# Setup MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['airbnb']

# Define the aggregation pipeline
pipeline = [
    {
        '$lookup': {
            'from': 'listings_with_calendar',
            'localField': 'id',
            'foreignField': '_id',
            'as': 'cal_docs'
        }
    },
    {
        '$lookup': {
            'from': 'reviews_t',
            'localField': 'id',
            'foreignField': 'listing_id',
            'as': 'reviews'
        }
    },
    {
        '$unwind': {
            'path': '$cal_docs',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$group': {
            '_id': '$_id',
            'id': {'$first': '$id'},
            'name': {'$first': '$name'},
            'host_id': {'$first': '$host_id'},
            'host_name': {'$first': '$host_name'},
            'neighbourhood_group': {'$first': '$neighbourhood_group'},
            'neighbourhood': {'$first': '$neighbourhood'},
            'latitude': {'$first': '$latitude'},
            'longitude': {'$first': '$longitude'},
            'room_type': {'$first': '$room_type'},
            'price': {'$first': '$price'},
            'minimum_nights': {'$first': '$minimum_nights'},
            'number_of_reviews': {'$first': '$number_of_reviews'},
            'last_review': {'$first': '$last_review'},
            'reviews_per_month': {'$first': '$reviews_per_month'},
            'calculated_host_listings_count': {'$first': '$calculated_host_listings_count'},
            'availability_365': {'$first': '$availability_365'},
            'number_of_reviews_ltm': {'$first': '$number_of_reviews_ltm'},
            'license': {'$first': '$license'},
            'average_price': {'$avg': '$cal_docs.average_price'},
            'first_available_date': {'$min': '$cal_docs.first_available_date'},
            'last_available_date': {'$max': '$cal_docs.last_available_date'},
            'reviews': {'$push': '$reviews'},
            'dates_list': {
                '$push': {
                    'date': '$cal_docs.dates_list.date',
                    'price': '$cal_docs.dates_list.price',
                    'minimum_nights': '$cal_docs.dates_list.minimum_nights',
                    'maximum_nights': '$cal_docs.dates_list.maximum_nights',
                    'available': '$cal_docs.dates_list.available'
                }
            }
        }
    },
    {
        '$out': 'listings_with_reviews_and_cal'
    }
]

# Execute the pipeline
db.listings_with_reviews_m.aggregate(pipeline)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'airbnb'), 'listings_with_reviews_m.head')


In [20]:
# Setup MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['airbnb']

# Fetch the processed documents
subset_docs = db.listings_with_reviews_and_cal.find({'id': {'$regex': '^111.*'}})

# Function to replace NaN with None and format datetime
def clean_document(doc):
    for key, value in doc.items():
        if isinstance(value, float) and math.isnan(value):
            doc[key] = None
        elif isinstance(value, ObjectId):
            doc[key] = str(value)
        elif isinstance(value, datetime.datetime):
            doc[key] = value.strftime('%Y-%m-%d')
        elif isinstance(value, list):
            for item in value:
                clean_document(item)
        elif isinstance(value, dict):
            clean_document(value)
    return doc

cleaned_documents = [clean_document(doc) for doc in subset_docs]




def write_dict_to_dir_json(data, dir, filename):
    # Creates a JSON file from dictionary data
    import os
    if not os.path.exists(dir):
        os.makedirs(dir)  # Ensure the directory exists
    full_path = f"{dir}/{filename}"
    with open(full_path, 'w') as fp:
        json.dump(data, fp, default=json_util.default)  # Use json_util.default to handle MongoDB specific types if needed

# Directory and filename
dir = 'OUTPUTS'
filename = 'listings_with_reviews_and_cal_subset_111.json'

# Call the function with correct parameters
write_dict_to_dir_json(subset_docs, dir, filename)

AttributeError: 'list' object has no attribute 'items'